## 准备数据

In [2]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [8]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [17]:
 


class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        self.W1 = tf.Variable(tf.random.normal([28*28, 100]),name='W1')
        self.b1 = tf.Variable(tf.zeros([100]), name='b1')
        # Define parameters and biases for the second dense layer
        self.W2 = tf.Variable(tf.random.normal([100, 10]), name='W1')
        self.b2 = tf.Variable(tf.zeros([10]), name='b2')
        ####################
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        logits = tf.reshape(x, [-1, 28*28])  
        logits = tf.matmul(logits, self.W1) + self.b1  
        logits = tf.nn.relu(logits)  
        logits = tf.matmul(logits, self.W2) + self.b2  
        ####################
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [18]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [22]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 9.973646 ; accuracy 0.6702333
epoch 1 : loss 9.951841 ; accuracy 0.67081666
epoch 2 : loss 9.930177 ; accuracy 0.67156667
epoch 3 : loss 9.908648 ; accuracy 0.6720667
epoch 4 : loss 9.887256 ; accuracy 0.6727
epoch 5 : loss 9.866002 ; accuracy 0.6731833
epoch 6 : loss 9.844883 ; accuracy 0.67368335
epoch 7 : loss 9.8239 ; accuracy 0.6742
epoch 8 : loss 9.803046 ; accuracy 0.67475
epoch 9 : loss 9.782321 ; accuracy 0.6752167
epoch 10 : loss 9.7617235 ; accuracy 0.6757
epoch 11 : loss 9.74125 ; accuracy 0.6763833
epoch 12 : loss 9.720902 ; accuracy 0.67681664
epoch 13 : loss 9.700678 ; accuracy 0.67733335
epoch 14 : loss 9.680575 ; accuracy 0.67773336
epoch 15 : loss 9.660594 ; accuracy 0.6781667
epoch 16 : loss 9.640731 ; accuracy 0.6785167
epoch 17 : loss 9.620993 ; accuracy 0.67905
epoch 18 : loss 9.601375 ; accuracy 0.67943335
epoch 19 : loss 9.58188 ; accuracy 0.6799
epoch 20 : loss 9.562502 ; accuracy 0.68041664
epoch 21 : loss 9.543241 ; accuracy 0.68075
epoch 22 : 